# Create manual eval

In [11]:
import pandas as pd
import pickle
from label.label_constants import *

split = 'test'
with open(f"../data/relation_extraction/model_data/{split}.pkl", 'rb') as fid:
    df = pickle.load(fid)
print(df.columns)
    
label_fns = [col for col in df.columns if col.endswith('lf')]
kb_labels = [col for col in df.columns if col.startswith('kb')]

df['label_fn_label'] = df.apply(lambda row: max([row[c] for c in label_fns]), axis=1)
df['kb_label'] = df.apply(lambda row: max([row[c] for c in kb_labels]), axis=1)
df.label_fn_label = df.label_fn_label.apply(lambda x: human_label(x, label_classes))
df.kb_label = df.kb_label.apply(lambda x: human_label(x, label_classes))

predictions = pd.read_pickle(f"../data/relation_extraction/{split}-model_best-predictions.pkl")
predictions['model_score'] = predictions.score.apply(lambda x: max(x))
predictions['sentence'] = predictions['text']
df = df.merge(predictions, how='left', on=['sentence', 'term_pair'])
df['model_label'] = df.predicted_relation.fillna('OTHER')
df['sort_value'] = (df.model_label != 'OTHER') | (df.gold_label != 'OTHER') | (~df.label_fn_label.isin(['OTHER', 'ABSTAIN'])) | (~df.kb_label.isin(['OTHER', 'ABSTAIN']))
df = df.sort_values(by = ['sort_value', 'model_score'], ascending=[False, True])
df = df[['textbook', 'chapter', 'section', 'sentence', 'term_pair', 'gold_label', 'model_label', 'model_score', 'label_fn_label', 'kb_label']]
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df[(df.model_label.isin(['HAS-PART/REGION', 'PART/REGION-OF'])) & (df.gold_label.isin(['OTHER']))].to_csv('fp.csv', index=False)
df[(df.gold_label.isin(['HAS-PART/REGION', 'PART/REGION-OF'])) & (df.model_label.isin(['OTHER']))].to_csv('fn.csv', index=False)
df.to_csv(f'{split}-manual_eval.csv', index=False)
df.tail()

Index(['index', 'chapter', 'section', 'section_name', 'sentence_number',
       'sentence', 'textbook', 'tokens', 'tagged_sentence', 'terms',
       'term_pair', 'term1', 'term2', 'term1_location', 'term2_location',
       'kb_bio101_ds_negative', 'term_part_of_speech', 'has_pattern_lf',
       'in_pattern_lf', 'poss_pattern_lf', 'of_pattern_lf',
       'contain_pattern_lf', 'consist_pattern_lf', 'partof_pattern_lf',
       'kb_bio101_ds_has_part', 'kb_bio101_ds_has_region',
       'kb_bio101_ds_taxonomy', 'kb_bio101_ds_synonym', 'isa_pattern_lf',
       'suchas_pattern_lf', 'including_pattern_lf', 'called_pattern_lf',
       'especially_pattern_lf', 'appo_pattern_lf', 'other_pattern_lf',
       'are_pattern_lf', 'symbolconj_pattern_lf', 'term_modifier_lf',
       'term_subset_lf', 'also_knownas_pattern_lf', 'parens_pattern_lf',
       'also_called_pattern_lf', 'plural_pattern_lf', 'majority_vote',
       'majority_vote_relation', 'label_model_labels', 'gold_label',
       'hard_label'

,textbook,chapter,section,sentence,term_pair,gold_label,model_label,model_score,label_fn_label,kb_label
298,Life_Biology,39.0,1.0,"Cells form tissues, tissues form organs, organ...","(form, organ system)",OTHER,OTHER,0.707857,ABSTAIN,OTHER
573,Life_Biology,39.0,2.0,Whatever force is pushing the system away from...,"(system, set point)",OTHER,OTHER,0.708087,ABSTAIN,ABSTAIN
1629,Life_Biology,39.0,4.0,Any heat the blood picks up from metabolically...,"(heat, gill)",OTHER,OTHER,0.708341,ABSTAIN,OTHER
1611,Life_Biology,39.0,4.0,Blood pumped from the fish heart goes directly...,"(fish heart, carbon dioxide)",OTHER,OTHER,0.709152,ABSTAIN,ABSTAIN
1628,Life_Biology,39.0,4.0,Any heat the blood picks up from metabolically...,"(blood, gill)",OTHER,OTHER,0.710774,ABSTAIN,OTHER


# Eval

In [56]:
import pickle
import pandas as pd
from ast import literal_eval

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

split = 'dev'

with open(f"../data/relation_extraction/model_data/{split}.pkl", 'rb') as fid:
    df = pickle.load(fid)
    
# load predictions
predictions = pd.read_pickle(f"../data/relation_extraction/model_data/{split}-model_best-predictions.pkl")
predictions['sentence'] = predictions['text']
predictions['model_label'] = predictions['predicted_relation']
df = df.merge(predictions, how='left', on=['sentence', 'term_pair'])


label_fns = [col for col in df.columns if col.endswith('lf')]
kb_labels = [col for col in df.columns if col.startswith('kb')]
df.label_model_labels = [tuple(x) for x in df.label_model_labels]
df['label_fn_label'] = df.apply(lambda row: max([row[c] for c in label_fns]), axis=1)
df['kb_label'] = df.apply(lambda row: max([row[c] for c in kb_labels]), axis=1)
df.label_fn_label = df.label_fn_label.apply(lambda x: human_label(x, label_classes))
df.kb_label = df.kb_label.apply(lambda x: human_label(x, label_classes))

In [57]:
def flip_tp(x, label):
    if x[label] == 'SUPERCLASS':
        return (x['term_pair'][1], x['term_pair'][0])
    else:
        return x['term_pair']
    
def rev_tps(tps):
    return set([(x[1], x[0]) for x in tps])

def term_eval(df, pred_col, label_col):
    df[pred_col] = df[pred_col].replace('ABSTAIN', 'OTHER')
    
    label_tps = df[~df[label_col].isin(['OTHER', 'ABSTAIN'])][['term_pair', label_col]]
    label_tps = set(label_tps.apply(lambda x: flip_tp(x, label_col), axis=1))
    pred_tps = df[~df[pred_col].isin(['OTHER', 'ABSTAIN'])][['term_pair', pred_col]]
    pred_tps = set(pred_tps.apply(lambda x: flip_tp(x, pred_col), axis=1))
    
    fp = pred_tps - label_tps 
    tp = pred_tps.intersection(label_tps)
    fn = label_tps - pred_tps
    
    print(f"Term Pair Precision: {(len(tp)) / (len(fp) + len(tp))}")
    print(f"Term Pair Recall: {(len(tp)) / (len(fn) + len(tp))}")
    
    return {'false_positives': fp, 'true_positives': tp, 'false_negatives': fn}

In [58]:
kb_ec = term_eval(df, 'kb_label', 'gold_label')

Term Pair Precision: 0.7226277372262774
Term Pair Recall: 0.24444444444444444


In [59]:
lfn_ec = term_eval(df, 'label_fn_label', 'gold_label')

Term Pair Precision: 0.6614173228346457
Term Pair Recall: 0.6222222222222222


In [60]:
pred_ec = term_eval(df, 'model_label', 'gold_label')

Term Pair Precision: 0.7356948228882834
Term Pair Recall: 0.6666666666666666


In [26]:
pred_ec['false_negatives']

{('actin', 'globular protein'),
 ('actin', 'protein'),
 ('activator', 'protein'),
 ('alpha tubulin', 'globular protein'),
 ('animal', 'eukaryote'),
 ('archaea', 'prokaryotic cell'),
 ('bacteria', 'prokaryotic cell'),
 ('bacterium', 'prokaryote'),
 ('beta tubulin', 'globular protein'),
 ('binary fission', 'cell division'),
 ('cadherin', 'protein'),
 ('cancer', 'disease'),
 ('cancer', 'uncontrolled cell growth'),
 ('cartilage', 'tissue'),
 ('cdk', 'protein'),
 ('cdk gene', 'gene'),
 ('cdk gene', 'proto oncogene'),
 ('cell', 'divide cell'),
 ('cell fractionation', 'process'),
 ('cell membrane', 'structure'),
 ('cell nucleus', 'nucleus'),
 ('cell nucleus', 'organelle'),
 ('cell organelle', 'structure'),
 ('cell wall', 'cellular structure'),
 ('cellulose', 'organic molecule'),
 ('cellulose', 'polysaccharide'),
 ('centrifugation', 'cell fractionation'),
 ('centriole', 'structure'),
 ('centrosome', 'cellular structure'),
 ('chlorophyll', 'pigment'),
 ('chloroplast', 'cellular structure'),
 ('

In [134]:
#fp_rev = set([(x[1], x[0]) for x in fp])
df[(df.term_pair.isin(lfn_ec['false_positives'])) & (df.label_fn_label > 0)].sample(10)

,textbook,chapter,sentence_id,text,tokens,term1,term1_location,term2,term2_location,term_pair,isa_pattern,suchas_pattern,including_pattern,called_pattern,especially_pattern,appo_pattern,other_pattern,are_pattern,whichis_pattern,term_part_of_speech,term_dep_role,term_modifier,term_subset,kb_bio101_ds_taxonomy,kb_bio101_ds_negative,nsubj_pattern,majority_vote,label_model_labels,gold_label,hard_label,soft_label,label,relation,prediction,score,hard_label_preds,label_fn_label
1199,OpenStax Biology 2e,4,4.104,Vesicles and Vacuoles Vesicles and vacuoles are membrane-bound sacs that function in storage and transport.,"[Vesicles, and, Vacuoles, , Vesicles, , and, , vacuoles, , are, membrane, -, bound, sacs, that, function, in, storage, and, transport, .]",vesicle,"(4, 5)",bind sacs,"(13, 15)","(vesicle, bind sacs)",-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,0,-1,-1,1,"(0.2784019554387749, 0.6092313182687287, 0.1123667262924964)",0,0,"[0.2784019554387749, 0.6092313182687287, 0.1123667262924964]",NaN,NaN,NaN,NaN,NaN,1
635,OpenStax Biology 2e,4,4.25,"People with celiac disease have an immune response to gluten, which is a protein in wheat, barley, and rye.","[People, with, celiac, disease, have, an, immune, response, to, gluten, ,, which, is, a, protein, in, wheat, ,, barley, ,, and, rye, .]",immune response,"(6, 8)",protein,"(14, 15)","(immune response, protein)",-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,0,0,-1,-1,"(0.35687528571904586, 0.507577300619062, 0.13554741366189202)",0,0,"[0.35687528571904586, 0.507577300619062, 0.13554741366189202]",0.0,OTHER,0.0,"[-0.03164100646972656, -3.545898914337158, -6.0735249519348145]",0.0,1
3687,OpenStax Biology 2e,10,3.29,"The Mitotic Phase The mitotic phase is a multistep process during which the duplicated chromosomes are aligned, separated, and move into two new, identical daughter cells.","[The, Mitotic, Phase, , The, mitotic, phase, is, a, multistep, process, during, which, the, duplicated, chromosomes, are, aligned, ,, separated, ,, and, move, into, two, new, ,, identical, daughter, cells, .]",mitotic phase,"(5, 7)",multistep process,"(9, 11)","(mitotic phase, multistep process)",1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,1,"(0.17890889831080264, 0.6571229401585241, 0.16396816153067328)",0,0,"[0.17890889831080264, 0.6571229401585241, 0.16396816153067328]",NaN,NaN,NaN,NaN,NaN,1
1206,OpenStax Biology 2e,4,4.104,Vesicles and Vacuoles Vesicles and vacuoles are membrane-bound sacs that function in storage and transport.,"[Vesicles, and, Vacuoles, , Vesicles, , and, , vacuoles, , are, membrane, -, bound, sacs, that, function, in, storage, and, transport, .]",vacuole,"(8, 9)",bind sacs,"(13, 15)","(vacuole, bind sacs)",-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,0,-1,-1,1,"(0.2784019554387749, 0.6092313182687287, 0.1123667262924964)",0,0,"[0.2784019554387749, 0.6092313182687287, 0.1123667262924964]",NaN,NaN,NaN,NaN,NaN,1
5090,OpenStax Biology 2e,10,5.23,"In addition to the cell-cycle regulatory proteins, any protein that influences the cycle can be altered in such a way as to override cell-cycle checkpoints.","[In, addition, to, the, cell, -, cycle, regulatory, proteins, ,, any, protein, that, influences, the, cycle, can, be, altered, in, such, a, way, as, to, override, cell, -, cycle, checkpoints, .]",cell cycle,"(4, 7)",cycle,"(15, 16)","(cell cycle, cycle)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,0,0,-1,-1,"(8.558480475088664e-05, 0.7054552993718903, 0.2944591158233588)",0,0,"[8.558480475088664e-05, 0.7054552993718903, 0.2944591158233588]",NaN,NaN,NaN,NaN,NaN,1
5012,OpenStax Biology 2e,10,5.13,"Proto-oncogenes are normal genes that, when mutated in certain ways, become oncogenes —genes that cause a cell to become cancerous.","[Proto, -, oncogenes, are, normal, genes, that, ,, when, mutated, in, certain, ways, ,, become, , oncogenes, —, genes, that, cause, a, cell, to, become, cancerous, .]",Proto oncogene,"(0, 3)",oncogene,"(16, 17)","(Proto oncogene, oncogene)",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,0,-1,-1,1,"